### 3.3 Pre-entrenamiento

En esta seccióon se utilizará un AE para pre-entrenar redes profundas. El efecto esperado es regularizar el modelo, posicionando el modelo de partida en una buena zona del espacio de parámetros. Se hace uso de las siguientes librerías:

In [1]:
from numpy.random import binomial
from keras.datasets import mnist
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
from keras.layers import Input, Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model, load_model, save_model, Sequential

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


Se carga y pre-procesa el data set de igual manera que en la secciones _3.1_ y _3.2_:

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Normalizacion de imagenes
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Transformación en vectores R^{784}
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Creación conjunto de validación
nval = 5000
x_val = x_train[-nval:]
y_val = y_train[-nval:]
x_train = x_train[:-nval]
y_train = y_train[:-nval]

# Transformación de salidas a probabilidades de activación
Y_train = np_utils.to_categorical(y_train, 10)
Y_val = np_utils.to_categorical(y_val, 10)
Y_test = np_utils.to_categorical(y_test, 10)

***a)*** Inicialmente se entrena una red FF para clasificar las imágenes de MNIST. Se emplea SGD básico con tasa de aprendizaje fija $\eta = 1$ y 50 epochs. Se utiliza una arquitectura $784 \times 1000 \times 1000 \times 10$ y funciones de activación sigmoidales. Se procede a determinar el error de clasificación alcanzado por el modelo en el conjunto de test, sin el empleo de pre-entrenamiento por medio de AE's.

In [3]:
input_img = Input(shape=(784,))

n_hidden_layer1 = 1000
activation_layer1 = 'sigmoid'
decoder_activation_1 = 'sigmoid'
n_hidden_layer2 = 1000
activation_layer2 = 'sigmoid'
decoder_activation_2 = 'sigmoid'
loss_ = 'binary_crossentropy'
optimizer_ = SGD(lr=1.0)
epochs_ = 50
batch_size_ = 25

model = Sequential()
model.add(Dense(n_hidden_layer1, activation=activation_layer1, input_shape=(784,)))
model.add(Dense(n_hidden_layer2, activation=activation_layer2))
model.add(Dense(10, activation='softmax'))
model.summary()

model.compile(optimizer=optimizer_, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, Y_train, epochs=epochs_, batch_size=25,shuffle=True, validation_data=(x_val, Y_val), verbose=1)
model.save('entrenamientos/SigmoidNet784x1000x1000x10-50epochs.h5')
#TRAINING CAN THEN BE RESUMED FROM THIS POINT :-)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_6 (Dense)              (None, 10)                10010     
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
_________________________________________________________________
Train on 55000 samples, validate on 5000 samples
Epoch 1/50
55000/55000 [==============================] - 30s - loss: 0.1519 - acc: 0.9492 - val_loss: 0.0508 - val_acc: 0.9834
Epoch 2/50
55000/55000 [==============================] - 29s - loss: 0.0581 - acc: 0.9803 - val_loss: 0.0382 - val_acc: 0.9874
Epoch 3/50
55000/55000 [==============================] - 29s - loss: 0.0481 - acc: 0.9839 

In [9]:
model1 = load_model('entrenamientos/SigmoidNet784x1000x1000x10-50epochs.h5');
model1.compile(optimizer=SGD(lr=1.0), loss='binary_crossentropy', metrics=['accuracy'])
loss,acc = model1.evaluate(x_test, Y_test, verbose=0)
print('Test loss: {0}'.format(loss)); print('Test accuracy: {0}'.format(acc))
print ('Error de clasificacion en conjunto de test:', (1 - acc) * 100,'%')

Test loss: 0.01269852604996704
Test accuracy: 0.9960399953842163
Error de clasificacion en conjunto de test: 0.396000461578 %


Así, se determina un error de clasificación alcanzado por el modelo en el conjunto de test igual a $0.396000461578\%$. Resulta ser un error bastante competitivo, implicando un desafío considerable de superar (experimentación realizada en ítems posteriores) por modelos pre-entrenados con AE's.  

***b)***

In [5]:
## PARAMETERS
n_hidden_layer1 = 1000
activation_layer1 = 'sigmoid'; decoder_activation_1 = 'sigmoid'
n_hidden_layer2 = 1000
activation_layer2 = 'sigmoid'; decoder_activation_2 = 'sigmoid'
loss_ = 'binary_crossentropy'
optimizer_ = SGD(lr=1.0)
epochs_ = 50
batch_size_ = 25

### AUTOENCODER 1
input_img1 = Input(shape=(784,))
encoded1 = Dense(n_hidden_layer1, activation=activation_layer1)(input_img1)
decoded1 = Dense(784, activation=decoder_activation_1)(encoded1)
autoencoder1 = Model(input=input_img1, output=decoded1)
encoder1 = Model(input=input_img1, output=encoded1)
autoencoder1.compile(optimizer=optimizer_, loss=loss_)
autoencoder1.fit(x_train, x_train, epochs=epochs_, batch_size=batch_size_, 
                 shuffle=True, validation_data=(x_val, x_val))
autoencoder1.save('entrenamientos/3_3/Sigmoid_autoencoder_layer1_ae.h5')
encoder1.save('entrenamientos/3_3/Sigmoid_encoder_layer1_ae.h5')

### AUTOENCODER 2
# FORWARD PASS DATA THROUGH FIRST ENCODER
x_train_encoded1 = encoder1.predict(x_train) 
x_val_encoded1 = encoder1.predict(x_val)
x_test_encoded1 = encoder1.predict(x_test)

input_img2 = Input(shape=(n_hidden_layer1,))
encoded2 = Dense(n_hidden_layer2, activation=activation_layer2)(input_img2)
decoded2 = Dense(n_hidden_layer1, activation=decoder_activation_2)(encoded2)
autoencoder2 = Model(input=input_img2, output=decoded2)
encoder2 = Model(input=input_img2, output=encoded2)
autoencoder2.compile(optimizer=optimizer_, loss=loss_)
autoencoder2.fit(x_train_encoded1, x_train_encoded1, epochs=epochs_, batch_size=batch_size_,
                 shuffle=True, validation_data=(x_val_encoded1, x_val_encoded1))
encoded_input2 = Input(shape=(n_hidden_layer2,))
autoencoder2.save('entrenamientos/3_3/Sigmoid_autoencoder_layer2_ae.h5')
encoder2.save('entrenamientos/3_3/Sigmoid_encoder_layer2_ae.h5')

### FINE TUNNING
model = Sequential()
model.add( Dense(n_hidden_layer1, activation=activation_layer1, input_shape=(784,)) )
model.layers[-1].set_weights( autoencoder1.layers[1].get_weights() )
model.add( Dense(n_hidden_layer2, activation=activation_layer2) )
model.layers[-1].set_weights( autoencoder2.layers[1].get_weights() )
model.add(Dense(10, activation='softmax'))
model.compile(optimizer=optimizer_, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, Y_train, epochs=20, batch_size=25, shuffle=True, validation_data=(x_val, Y_val))
model.save('entrenamientos/3_3/Sigmoid_784x1000x1000x10_finetunning_ae.h5')

Train on 55000 samples, validate on 5000 samples
Epoch 1/50
55000/55000 [==============================] - 37s - loss: 0.2369 - val_loss: 0.1995
Epoch 2/50
55000/55000 [==============================] - 27s - loss: 0.1802 - val_loss: 0.1655
Epoch 3/50
55000/55000 [==============================] - 27s - loss: 0.1561 - val_loss: 0.1478
Epoch 4/50
55000/55000 [==============================] - 27s - loss: 0.1415 - val_loss: 0.1359
Epoch 5/50
55000/55000 [==============================] - 27s - loss: 0.1312 - val_loss: 0.1272
Epoch 6/50
55000/55000 [==============================] - 27s - loss: 0.1235 - val_loss: 0.1205
Epoch 7/50
55000/55000 [==============================] - 27s - loss: 0.1175 - val_loss: 0.1152
Epoch 8/50
55000/55000 [==============================] - 27s - loss: 0.1127 - val_loss: 0.1108
Epoch 9/50
55000/55000 [==============================] - 27s - loss: 0.1087 - val_loss: 0.1072
Epoch 10/50
55000/55000 [==============================] - 27s - loss: 0.1053 - val_los

55000/55000 [==============================] - 30s - loss: 0.5588 - val_loss: 0.5592
Epoch 35/50
55000/55000 [==============================] - 30s - loss: 0.5587 - val_loss: 0.5591
Epoch 36/50
55000/55000 [==============================] - 30s - loss: 0.5586 - val_loss: 0.5590
Epoch 37/50
55000/55000 [==============================] - 30s - loss: 0.5585 - val_loss: 0.5589
Epoch 38/50
55000/55000 [==============================] - 30s - loss: 0.5584 - val_loss: 0.5588
Epoch 39/50
55000/55000 [==============================] - 30s - loss: 0.5584 - val_loss: 0.5588
Epoch 40/50
55000/55000 [==============================] - 30s - loss: 0.5583 - val_loss: 0.5587
Epoch 41/50
55000/55000 [==============================] - 30s - loss: 0.5582 - val_loss: 0.5586
Epoch 42/50
55000/55000 [==============================] - 30s - loss: 0.5582 - val_loss: 0.5586
Epoch 43/50
55000/55000 [==============================] - 30s - loss: 0.5581 - val_loss: 0.5585
Epoch 44/50
55000/55000 [=================

***c)***

In [6]:
def gaussian_noise(X_train, X_val, X_test, devst=0.1):
    noise_mask = devst*np.random.standard_normal(size=X_train.shape)
    X_train_noisy = X_train + noise_mask
    noise_mask = devst*np.random.standard_normal(size=X_val.shape)
    X_val_noisy = X_val + noise_mask
    noise_mask = devst*np.random.standard_normal(size=X_test.shape)
    X_test_noisy = X_test + noise_mask
    return X_train_noisy, X_val_noisy, X_test_noisy

In [ ]:
## PARAMETERS
n_hidden_layer1 = 1000
activation_layer1 = 'sigmoid'; decoder_activation_1 = 'sigmoid'
n_hidden_layer2 = 1000
activation_layer2 = 'sigmoid'; decoder_activation_2 = 'sigmoid'
loss_ = 'binary_crossentropy'
optimizer_ = SGD(lr=1.0)
epochs_ = 50
batch_size_ = 25

# Data with gaussian noise
x_train_noisy, x_val_noisy, x_test_noisy = gaussian_noise(x_train, x_val, x_test)

### DENOISING AUTOENCODER 1
input_img1 = Input(shape=(784,))
encoded1 = Dense(n_hidden_layer1, activation=activation_layer1)(input_img1)
decoded1 = Dense(784, activation=decoder_activation_1)(encoded1)
autoencoder1 = Model(input=input_img1, output=decoded1)
encoder1 = Model(input=input_img1, output=encoded1)
autoencoder1.compile(optimizer=optimizer_, loss=loss_)
autoencoder1.fit(x_train_noisy, x_train, epochs=epochs_, batch_size=batch_size_, 
                 shuffle=True, validation_data=(x_val_noisy, x_val))
autoencoder1.save('entrenamientos/3_3/Sigmoid_autoencoder_layer1_dae.h5')
encoder1.save('entrenamientos/3_3/Sigmoid_encoder_layer1_dae.h5')

### DENOISING AUTOENCODER 2
# FORWARD PASS DATA THROUGH FIRST ENCODER
x_train_encoded1 = encoder1.predict(x_train) 
x_val_encoded1 = encoder1.predict(x_val)
x_test_encoded1 = encoder1.predict(x_test)
# adding gaussian noise also to encoded data
x_train_noisy_encoded1, x_val_noisy_encoded1, x_test_noisy_encoded1 = gaussian_noise(x_train_encoded1, x_val_encoded1,
                                                                                     x_test_encoded1)
input_img2 = Input(shape=(n_hidden_layer1,))
encoded2 = Dense(n_hidden_layer2, activation=activation_layer2)(input_img2)
decoded2 = Dense(n_hidden_layer1, activation=decoder_activation_2)(encoded2)
autoencoder2 = Model(input=input_img2, output=decoded2)
encoder2 = Model(input=input_img2, output=encoded2)
autoencoder2.compile(optimizer=optimizer_, loss=loss_)
autoencoder2.fit(x_train_noisy_encoded1, x_train_encoded1, epochs=epochs_, batch_size=batch_size_,
                 shuffle=True, validation_data=(x_val_noisy_encoded1, x_val_encoded1))
encoded_input2 = Input(shape=(n_hidden_layer2,))
autoencoder2.save('entrenamientos/3_3/Sigmoid_autoencoder_layer2_dae.h5')
encoder2.save('entrenamientos/3_3/Sigmoid_encoder_layer2_dae.h5')

### FINE TUNNING
model = Sequential()
model.add( Dense(n_hidden_layer1, activation=activation_layer1, input_shape=(784,)) )
model.layers[-1].set_weights( autoencoder1.layers[1].get_weights() )
model.add( Dense(n_hidden_layer2, activation=activation_layer2) )
model.layers[-1].set_weights( autoencoder2.layers[1].get_weights() )
model.add(Dense(10, activation='softmax'))
model.compile(optimizer=optimizer_, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, Y_train, epochs=20, batch_size=25, shuffle=True, validation_data=(x_val, Y_val))
model.save('entrenamientos/3_3/Sigmoid_784x1000x1000x10_finetunning_dae.h5')

Train on 55000 samples, validate on 5000 samples
Epoch 1/50
55000/55000 [==============================] - 31s - loss: 0.2371 - val_loss: 0.1999
Epoch 2/50
55000/55000 [==============================] - 27s - loss: 0.1805 - val_loss: 0.1658
Epoch 3/50
55000/55000 [==============================] - 26s - loss: 0.1565 - val_loss: 0.1483
Epoch 4/50
55000/55000 [==============================] - 26s - loss: 0.1421 - val_loss: 0.1366
Epoch 5/50
55000/55000 [==============================] - 26s - loss: 0.1320 - val_loss: 0.1280
Epoch 6/50
55000/55000 [==============================] - 26s - loss: 0.1245 - val_loss: 0.1215
Epoch 7/50
55000/55000 [==============================] - 26s - loss: 0.1185 - val_loss: 0.1162
Epoch 8/50
55000/55000 [==============================] - 26s - loss: 0.1137 - val_loss: 0.1119
Epoch 9/50
55000/55000 [==============================] - 26s - loss: 0.1098 - val_loss: 0.1083
Epoch 10/50
55000/55000 [==============================] - 26s - loss: 0.1065 - val_los

55000/55000 [==============================] - 31s - loss: 0.5602 - val_loss: 0.5606
Epoch 35/50
55000/55000 [==============================] - 30s - loss: 0.5601 - val_loss: 0.5605
Epoch 36/50
55000/55000 [==============================] - 30s - loss: 0.5600 - val_loss: 0.5604
Epoch 37/50
55000/55000 [==============================] - 30s - loss: 0.5599 - val_loss: 0.5603
Epoch 38/50
55000/55000 [==============================] - 31s - loss: 0.5599 - val_loss: 0.5603
Epoch 39/50
55000/55000 [==============================] - 31s - loss: 0.5598 - val_loss: 0.5602
Epoch 40/50
55000/55000 [==============================] - 31s - loss: 0.5597 - val_loss: 0.5601
Epoch 41/50
55000/55000 [==============================] - 31s - loss: 0.5597 - val_loss: 0.5601
Epoch 42/50
55000/55000 [==============================] - 31s - loss: 0.5596 - val_loss: 0.5600
Epoch 43/50
55000/55000 [==============================] - 31s - loss: 0.5595 - val_loss: 0.5599
Epoch 44/50
55000/55000 [=================

***d)***

_i) Función de activación ReLu:_

In [ ]:
## PARAMETERS
n_hidden_layer1 = 1000
activation_layer1 = 'relu'; decoder_activation_1 = 'sigmoid'
n_hidden_layer2 = 1000
activation_layer2 = 'relu'; decoder_activation_2 = 'sigmoid'
loss_ = 'binary_crossentropy'
optimizer_ = SGD(lr=1.0)
epochs_ = 50
batch_size_ = 25

### AUTOENCODER 1
input_img1 = Input(shape=(784,))
encoded1 = Dense(n_hidden_layer1, activation=activation_layer1)(input_img1)
decoded1 = Dense(784, activation=decoder_activation_1)(encoded1)
autoencoder1 = Model(input=input_img1, output=decoded1)
encoder1 = Model(input=input_img1, output=encoded1)
autoencoder1.compile(optimizer=optimizer_, loss=loss_)
autoencoder1.fit(x_train, x_train, epochs=epochs_, batch_size=batch_size_, 
                 shuffle=True, validation_data=(x_val, x_val))
autoencoder1.save('entrenamientos/3_3/Relu_autoencoder_layer1_ae.h5')
encoder1.save('entrenamientos/3_3/Relu_encoder_layer1_ae.h5')

### AUTOENCODER 2
# FORWARD PASS DATA THROUGH FIRST ENCODER
x_train_encoded1 = encoder1.predict(x_train) 
x_val_encoded1 = encoder1.predict(x_val)
x_test_encoded1 = encoder1.predict(x_test)

input_img2 = Input(shape=(n_hidden_layer1,))
encoded2 = Dense(n_hidden_layer2, activation=activation_layer2)(input_img2)
decoded2 = Dense(n_hidden_layer1, activation=decoder_activation_2)(encoded2)
autoencoder2 = Model(input=input_img2, output=decoded2)
encoder2 = Model(input=input_img2, output=encoded2)
autoencoder2.compile(optimizer=optimizer_, loss=loss_)
autoencoder2.fit(x_train_encoded1, x_train_encoded1, epochs=epochs_, batch_size=batch_size_,
                 shuffle=True, validation_data=(x_val_encoded1, x_val_encoded1))
encoded_input2 = Input(shape=(n_hidden_layer2,))
autoencoder2.save('entrenamientos/3_3/Relu_autoencoder_layer2_ae.h5')
encoder2.save('entrenamientos/3_3/Relu_encoder_layer2_ae.h5')

### FINE TUNNING
model = Sequential()
model.add( Dense(n_hidden_layer1, activation=activation_layer1, input_shape=(784,)) )
model.layers[-1].set_weights( autoencoder1.layers[1].get_weights() )
model.add( Dense(n_hidden_layer2, activation=activation_layer2) )
model.layers[-1].set_weights( autoencoder2.layers[1].get_weights() )
model.add(Dense(10, activation='softmax'))
model.compile(optimizer=optimizer_, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, Y_train, epochs=20, batch_size=25, shuffle=True, validation_data=(x_val, Y_val))
model.save('entrenamientos/3_3/Relu_784x1000x1000x10_finetunning_ae.h5')

Train on 55000 samples, validate on 5000 samples
Epoch 1/50
53375/55000 [============================>.] - ETA: 1s - loss: 0.1763

_ii) Función de activación Tanh:_

In [ ]:
## PARAMETERS
n_hidden_layer1 = 1000
activation_layer1 = 'tanh'; decoder_activation_1 = 'sigmoid'
n_hidden_layer2 = 1000
activation_layer2 = 'tanh'; decoder_activation_2 = 'sigmoid'
loss_ = 'binary_crossentropy'
optimizer_ = SGD(lr=1.0)
epochs_ = 50
batch_size_ = 25

### AUTOENCODER 1
input_img1 = Input(shape=(784,))
encoded1 = Dense(n_hidden_layer1, activation=activation_layer1)(input_img1)
decoded1 = Dense(784, activation=decoder_activation_1)(encoded1)
autoencoder1 = Model(input=input_img1, output=decoded1)
encoder1 = Model(input=input_img1, output=encoded1)
autoencoder1.compile(optimizer=optimizer_, loss=loss_)
autoencoder1.fit(x_train, x_train, epochs=epochs_, batch_size=batch_size_, 
                 shuffle=True, validation_data=(x_val, x_val))
autoencoder1.save('entrenamientos/3_3/Tanh_autoencoder_layer1_ae.h5')
encoder1.save('entrenamientos/3_3/Tanh_encoder_layer1_ae.h5')

### AUTOENCODER 2
# FORWARD PASS DATA THROUGH FIRST ENCODER
x_train_encoded1 = encoder1.predict(x_train) 
x_val_encoded1 = encoder1.predict(x_val)
x_test_encoded1 = encoder1.predict(x_test)

input_img2 = Input(shape=(n_hidden_layer1,))
encoded2 = Dense(n_hidden_layer2, activation=activation_layer2)(input_img2)
decoded2 = Dense(n_hidden_layer1, activation=decoder_activation_2)(encoded2)
autoencoder2 = Model(input=input_img2, output=decoded2)
encoder2 = Model(input=input_img2, output=encoded2)
autoencoder2.compile(optimizer=optimizer_, loss=loss_)
autoencoder2.fit(x_train_encoded1, x_train_encoded1, epochs=epochs_, batch_size=batch_size_,
                 shuffle=True, validation_data=(x_val_encoded1, x_val_encoded1))
encoded_input2 = Input(shape=(n_hidden_layer2,))
autoencoder2.save('entrenamientos/3_3/Tanh_autoencoder_layer2_ae.h5')
encoder2.save('entrenamientos/3_3/Tanh_encoder_layer2_ae.h5')

### FINE TUNNING
model = Sequential()
model.add( Dense(n_hidden_layer1, activation=activation_layer1, input_shape=(784,)) )
model.layers[-1].set_weights( autoencoder1.layers[1].get_weights() )
model.add( Dense(n_hidden_layer2, activation=activation_layer2) )
model.layers[-1].set_weights( autoencoder2.layers[1].get_weights() )
model.add(Dense(10, activation='softmax'))
model.compile(optimizer=optimizer_, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, Y_train, epochs=20, batch_size=25, shuffle=True, validation_data=(x_val, Y_val))
model.save('entrenamientos/3_3/Tanh_784x1000x1000x10_finetunning_ae.h5')